In [ ]:
!nvidia-smi

In [2]:
import torch
from torch.utils.data import Subset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

import numpy as np

import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from tqdm import tqdm

device: cuda


In [3]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
def get_subset_indices(dataset, percentage, seed=0):
  rng = np.random.RandomState(seed)
  # data = dataset.data
  targets = np.array(dataset.targets)
  num_classes = len(np.unique(targets))
  num_samples_per_class = int(percentage*len(dataset)/num_classes)

  indices = []

  for c in range(num_classes):
    class_indices = (targets == c).nonzero()[0]
    indices.extend(
        list(rng.choice(class_indices, size=num_samples_per_class, replace=False))
    )
  return indices

In [ ]:
GID_data_path = "/home/hz271/PyTorch-StudioGAN/biggan_images/samples/CIFAR10-BigGAN-DiffAug-train-2022_02_11_07_23_15/fake"

# Datasets
generated_train_dataset = torchvision.datasets.ImageFolder(
    root=GID_data_path,
    transform=transform_train
)

original_train_dataset = torchvision.datasets.CIFAR10(
    root='/home/hz271/OOD/Saved/data/', train=True, download=True, transform=transform_train)

# Concatenate
train_set = torch.utils.data.ConcatDataset(
    [generated_train_dataset, original_train_dataset]
)

test_set = torchvision.datasets.CIFAR10(
    root='/home/hz271/OOD/Saved/data/', train=False, download=True, transform=transform_test)